# SMA Assignment 2
## Group Members: Cherry, Sagar, Amy, Hope, Cory

### Twitter Scraper

In [ ]:
import time
import urllib
import http
import pandas as pd
from copy import copy
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

def get_comment_and_link(x, driver):
    try:
        link = x.find_element_by_tag_name('a').get_attribute('href')
        hover = ActionChains(driver).move_to_element(x)
        hover.perform()
        time.sleep(1)
        comment_count = x.find_elements_by_class_name('-V_eO')[1].find_element_by_tag_name('span').text
        return (link, comment_count)
    except IndexError:
        return (None, None)
    
def scrape_links_and_comments(n_scrape=0):
    n_scrape = n_scrape
    driver = webdriver.Chrome('/Users/Cory/Desktop/chromedriver')  # Optional argument, if not specified will search path.
    driver.get('https://www.instagram.com');
    
    # Get to Instagram and click the login box
    time.sleep(1) # Let the user actually see something!
    login = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div[2]/p/a')
    login.click()
    time.sleep(1)
    
    # Log in - Insert your username and password below
    user_box = driver.find_element_by_name('username')
    user_box.send_keys('username')
    pass_box = driver.find_element_by_name('password')
    pass_box.send_keys('password')
    time.sleep(1)
    pass_box.submit()
    time.sleep(3)
    
    # How to actually scrape
    target = 'natgeo'
    driver.get('https://www.instagram.com/' + target +'/')
    
    # Scrolling up and down
    n_scrape2 = copy(n_scrape) - 12
    series_of_links = []
    while n_scrape2 > 0:
        series_of_pictures = None
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # One scroll down catches 12 images
        time.sleep(5)
        series_of_pictures = pd.Series(driver.find_elements_by_css_selector('div.v1Nh3.kIKUG._bz0w')[:-12]).apply(get_comment_and_link, args=(driver,))
        shortlist_links = series_of_pictures.tolist()
        for i in shortlist_links:
            series_of_links.append(i)
        n_scrape2 -= 12
        time.sleep(1)
    links = set(series_of_links)

    return links

In [ ]:
a = scrape_links_and_comments(1000)
df = pd.DataFrame(a)

df.to_csv("~/Downloads/insta-1.csv") # This can be fed ahead into Task A

In [ ]:
# From the scraper I ran earlier, we already have the comment counts and CSS selector links for our instagram posts

df = pd.read_csv('~/Downloads/insta-1.csv')
df = df.dropna(how = 'any', axis = 0)
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

pics = []
likes = []
captions = []

links = df['0'].tolist()

driver = webdriver.Chrome('/Users/Cory/Desktop/chromedriver')  # Optional argument, if not specified will search path.

for i, link in enumerate(links):
    driver.get(link)
    try:
        picture = driver.find_element_by_class_name('KL4Bh').find_element_by_tag_name('img').get_attribute('src')
        pics.append(picture)
        
        like_count = driver.find_element_by_class_name('zV_Nj').find_element_by_tag_name('span').text
        likes.append(like_count)
        
        caption = driver.find_element_by_class_name('C4VMK').find_element_by_tag_name('span').text
        captions.append(caption)

        time.sleep(2)
    except NoSuchElementException:
        pics.append(None)
        likes.append(None)
        captions.append(None)
        time.sleep(1)
        
df['pic_link'] = pics
df['likes'] = likes
df['caption'] = captions
df = df.rename({'0':'link','1':'comments'}, axis=1)
df = df.dropna(how = 'any', axis = 0)
df.to_csv("~/Downloads/natgeo-insta-final.csv")

### Task A

In [ ]:
import io
import os
from google.cloud.vision import types
from google.cloud import vision
import numpy as np
import urllib
import pandas as pd

In [ ]:
## Calling the API using key
client = vision.ImageAnnotatorClient.from_service_account_file('C://Users//cherr//Downloads//SocialMedia-48509d145dcb.json')

In [ ]:
## Reading the data
data = pd.read_csv('insta-1.csv', header = None)

In [ ]:
## Cleaning the data
data = data.rename(index=str, columns={0: "index", 1: "URL"})
data.dropna(inplace = True)
data.shape

In [ ]:
def image_tags(url):
    resp = urllib.urlopen(url)
    content = resp.read()
    image_tags = []
    image = vision.types.Image(content=content)
    response = client.label_detection(image=image)
    labels = response.label_annotations
    for label in labels:
        image_tags.append(label.description)
    return(image_tags)

In [ ]:
data['google_image_tags'] = data['URL'].map(image_tags)

In [ ]:
data.to_csv('image_tags.csv')

### Task B

In [8]:
df = pd.read_csv('image_tags.csv')
df.head(2)

,Unnamed: 0,Unnamed: 0.1,link,comments,pic_link,likes,caption,google_image_tags
0,0,0,https://www.instagram.com/p/Bs7tibqDpOT/,"1,160",https://scontent-dfw5-1.cdninstagram.com/vp/ba...,"240,900","Photo by @noralorek | Peter Mandela, a 12-year...","[u'Brown', u'Standing', u'Tree', u'Fun', u'Foo..."
1,1,1,https://www.instagram.com/p/BrKZ6ttg0W7/,"1,125",https://scontent-dfw5-1.cdninstagram.com/vp/17...,"318,788",Photo by @BrianSkerry | Researcher Enric Sala ...,"[u'Rock', u'Earth', u'Circle', u'World']"


In [9]:
df.shape

(874, 8)

In [10]:
# clean the dataframe, remove unnecessary columns
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)

In [11]:
# the list is read as string in the google_image_tags column, lets correct this
from ast import literal_eval
df['google_image_tags'] = df['google_image_tags'].map(lambda x: literal_eval(x))

df.head(2)

,link,comments,pic_link,likes,caption,google_image_tags
0,https://www.instagram.com/p/Bs7tibqDpOT/,"1,160",https://scontent-dfw5-1.cdninstagram.com/vp/ba...,"240,900","Photo by @noralorek | Peter Mandela, a 12-year...","[Brown, Standing, Tree, Fun, Footwear, Grass, ..."
1,https://www.instagram.com/p/BrKZ6ttg0W7/,"1,125",https://scontent-dfw5-1.cdninstagram.com/vp/17...,"318,788",Photo by @BrianSkerry | Researcher Enric Sala ...,"[Rock, Earth, Circle, World]"


In [12]:
# convert the list of words into a continuous string
df['google_labels'] = df['google_image_tags'].map(lambda x: " ".join(x))
df.head(2)

,link,comments,pic_link,likes,caption,google_image_tags,google_labels
0,https://www.instagram.com/p/Bs7tibqDpOT/,"1,160",https://scontent-dfw5-1.cdninstagram.com/vp/ba...,"240,900","Photo by @noralorek | Peter Mandela, a 12-year...","[Brown, Standing, Tree, Fun, Footwear, Grass, ...",Brown Standing Tree Fun Footwear Grass Walking...
1,https://www.instagram.com/p/BrKZ6ttg0W7/,"1,125",https://scontent-dfw5-1.cdninstagram.com/vp/17...,"318,788",Photo by @BrianSkerry | Researcher Enric Sala ...,"[Rock, Earth, Circle, World]",Rock Earth Circle World


In [13]:
# define a new column order and remove pic_link
column_order = ['link', 'likes', 'comments', 'caption', 'google_labels']
df2 = df.loc[:, column_order]

df2.head(2)

,link,likes,comments,caption,google_labels
0,https://www.instagram.com/p/Bs7tibqDpOT/,"240,900","1,160","Photo by @noralorek | Peter Mandela, a 12-year...",Brown Standing Tree Fun Footwear Grass Walking...
1,https://www.instagram.com/p/BrKZ6ttg0W7/,"318,788","1,125",Photo by @BrianSkerry | Researcher Enric Sala ...,Rock Earth Circle World


### Cleaning the caption column

Let's remove the part of the text before the '|' character.

In [14]:
import re
from string import punctuation

import nltk
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

def remove_char(s):
    s = re.sub(".*\|\s", "", s)
    return s

df2['caption_new'] = df2['caption'].map(lambda x: remove_char(x))

### Lemmatization

In [22]:
import nltk
from nltk.tokenize import word_tokenize

# lemmatization
from nltk.corpus import wordnet

from nltk.stem import WordNetLemmatizer
word_lemm = WordNetLemmatizer()

def convert_to_valid_pos(x):
    
    x = x[0].upper() # extract first character of the POS tag
    
    # define mapping for the tag to correct tag.
    tag_dict = {"J": wordnet.ADJ,
               "N": wordnet.NOUN,
               "R": wordnet.ADV,
               "V": wordnet.VERB}
    
    return tag_dict.get(x, wordnet.NOUN)

def lemmatize_text(s):
    pos_tagged_text = nltk.pos_tag(word_tokenize(s.lower()))
    
    lemm_list = []

    for (word, tag) in pos_tagged_text:
        lemm_list.append(word_lemm.lemmatize(word, pos = convert_to_valid_pos(tag)))


    lemm_text = " ".join(lemm_list)
    return lemm_text

df2['caption_new'] = df2['caption_new'].map(lemmatize_text)

### Remove anomalous strings from the text

In [27]:
from string import punctuation
from nltk.corpus import stopwords # store english stopwords in a list
en_stopwords = stopwords.words('english')

def remove_anomalies(s):
    
    tokens = word_tokenize(s)
    
    # urls
    weblinks = [w for w in tokens if ".co.uk" in w] + [w for w in tokens if ".com" in w] + [w for w in tokens if "www" in w]
    weblinks = list(set(weblinks)) # remove duplicates from weblinks
    
    # numbers
    numbers = []
    for x in tokens:
        if len(re.findall('.*[0-9]+.*', x)) > 0:
            numbers.append(re.findall('.*[0-9]+.*', x)[0])
        else:
            numbers.append(np.nan)
    
    numbers = pd.Series(numbers)
    numbers = numbers[~numbers.isnull()].tolist()
    
    # stars
    stars = []
    for x in tokens:
        if len(re.findall('.*[\*]+.*', x)) > 0:
            stars.append(re.findall('.*[\*]+,*', x)[0])
        else:
            stars.append(0)
        
    stars = pd.Series(stars)   
    stars = stars[stars != 0].tolist()
    
    #stopwords
    global en_stopwords
    
    answer = " ".join([w for w in tokens if (w not in en_stopwords) & (w not in weblinks) & (w not in numbers) & (w not in stars)])
    
    for l in punctuation:
        answer = answer.replace(l, "")
    
    return answer

df2['caption_new'] = df2['caption_new'].map(remove_anomalies)

In [28]:
# create a new final dataframe that will be used for the analyses ahead.
df_final = df2.loc[:, ['link', 'google_labels', 'caption_new', 'likes', 'comments']]

In [29]:
# correct likes to remove commas and convert to integers
df_final['likes'] = df_final['likes'].map(lambda x: float(re.sub(",","",x)))

In [33]:
# remove commas from the comments column
df_final['comments'] = df_final['comments'].map(lambda x: re.sub(",", "", str(x)))

# some comment rows have values such as 13k etc. we need to correct these
def correct_k_values(s):
    if "k" in s:
        s = re.sub("k", "", s)
        s = str(float(s)*1000)
        return s
    else:
        return s
    
df_final['comments'] = df_final['comments'].map(lambda x: correct_k_values(x))

# convert to numbers
df_final['comments'] = df_final['comments'].map(lambda x: float(x))

In [34]:
# add a new column that takes both captions and labels
# we need to combine the captions and labels into a single column
df_final['caption_labels'] = df_final['caption_new'] + " " + df_final['google_labels']
df_final.head()

,link,google_labels,caption_new,likes,comments,caption_labels
0,https://www.instagram.com/p/Bs7tibqDpOT/,Brown Standing Tree Fun Footwear Grass Walking...,peter mandela refugee south sudan living ugan...,240900.0,1160.0,peter mandela refugee south sudan living ugan...
1,https://www.instagram.com/p/BrKZ6ttg0W7/,Rock Earth Circle World,researcher enric sala hovers giant circularsh...,318788.0,1125.0,researcher enric sala hovers giant circularsh...
2,https://www.instagram.com/p/BpjB91kDCdP/,Natural landscape Nature Natural environment T...,first assignment natgeo eleven year ago take...,341539.0,658.0,first assignment natgeo eleven year ago take...
3,https://www.instagram.com/p/BqscPhdh2M4/,Electronic instrument Recording Musician Broad...,part commitment enhance learn education techno...,100086.0,270.0,part commitment enhance learn education techno...
4,https://www.instagram.com/p/BrYxkCxAq16/,Green Cactus Plant Grass Prickly pear Aerial p...,believe flamingos withstand boil water tempera...,440059.0,1577.0,believe flamingos withstand boil water tempera...


## Model using just google labels
First we just need to use the google labels to predict engagement.

### Standardize the comments and likes
Engagement is the weighted score of comments and likes and is defined as 0.6 * comments + 0.4 * likes.

In [35]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

# standardize the likes and comments
df_final_std = std_scaler.fit_transform(df_final.loc[:, ['likes', 'comments']])
engagement_score = df_final_std[:, 0]*0.4 + df_final_std[:, 1]*0.6

# in case we want a dataframe back
feature_names = ['likes', 'comments', 'engagement']

In [67]:
df_like_comment_engg = pd.DataFrame(np.c_[df_final_std, engagement_score], columns = ['likes', 'comments', 'engagement'])

df_final_final = pd.concat([df_final.drop(['likes', 'comments'], axis = 1), df_like_comment_engg], axis = 1)

### Convert the labels into bag of words

In [36]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Calculate the frequencies of words using the TfidfTransformer
X = np.array(df_final.loc[:, 'google_labels'])
y = engagement_score

# split into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Convert the arrays into a presence/absence matrix
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_features = 200)
X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

# Fit the model
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators = 50, random_state = 42, max_features = 1)
rf_model.fit(X_train_counts, y_train)

predicted = rf_model.predict(X_train_counts)

# print the accuracies
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, predicted)))

Train RMSE: 0.2841905074541063


In [37]:
# cross validation scores
from sklearn.model_selection import cross_val_score

cross_val_scores_rf = cross_val_score(rf_model, 
                                      X_train_counts,
                                      y_train, 
                                      cv = 5,
                                      verbose = 2,
                                      n_jobs = -1,
                                      scoring = "neg_mean_squared_error")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.4s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.4s finished


In [38]:
-1*np.mean(cross_val_scores_rf)

0.564885262955436

In [39]:
# test error
predicted = rf_model.predict(X_test_counts)

# print the accuracies
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, predicted)))

Test RMSE: 0.8888911613936642


### Model using just captions
Next let's use the captions to predict engagement.

In [40]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Calculate the frequencies of words using the TfidfTransformer
X = np.array(df_final.loc[:, 'caption_new'])
y = engagement_score

# split into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Convert the arrays into a presence/absence matrix
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_features = 200)
X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

# Fit the model
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 42, max_features = 1)
rf_model.fit(X_train_counts, y_train)

predicted = rf_model.predict(X_train_counts)

# print the accuracies
print("RMSE:", np.sqrt(mean_squared_error(y_train, predicted)))

RMSE: 0.2597829648368051


In [41]:
# cross validation scores
from sklearn.model_selection import cross_val_score

cross_val_scores_rf = cross_val_score(rf_model, 
                                      X_train_counts,
                                      y_train, 
                                      cv = 5,
                                      verbose = 2,
                                      n_jobs = -1,
                                      scoring = "neg_mean_squared_error")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.3s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


In [42]:
-1*np.mean(cross_val_scores_rf)

0.5895979105826803

In [43]:
# test error
predicted = rf_model.predict(X_test_counts)

# print the accuracies
print("Train RMSE:", np.sqrt(mean_squared_error(y_test, predicted)))

Train RMSE: 1.0055770994487159


So we see that using just google labels leads to a better accuracy than using captions of the images that the photographers have provided.

### Model using both captions and labels

In [44]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Calculate the frequencies of words using the TfidfTransformer
X = np.array(df_final.loc[:, 'caption_labels'])
y = engagement_score

# split into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Convert the arrays into a presence/absence matrix
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_features = 200)
X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

# Fit the model
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 42, max_features = 4)
rf_model.fit(X_train_counts, y_train)

predicted = rf_model.predict(X_train_counts)

# print the accuracies
print("RMSE:", np.sqrt(mean_squared_error(y_train, predicted)))

RMSE: 0.2529394733443292


In [45]:
# cross validation scores
from sklearn.model_selection import cross_val_score

cross_val_scores_rf = cross_val_score(rf_model, 
                                      X_train_counts,
                                      y_train, 
                                      cv = 5,
                                      verbose = 2,
                                      n_jobs = -1,
                                      scoring = "neg_mean_squared_error")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


In [46]:
-1*np.mean(cross_val_scores_rf)

0.5562468050831829

In [47]:
# test error
predicted = rf_model.predict(X_test_counts)

# print the accuracies
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, predicted)))

Test RMSE: 0.9809533488836648


We see that we are better off using just google labels for the prediction task. The RMSE is the least for that case.

### Task C

Perform topic modeling (LDA) on the original image labels. Choose an appropriate number of topics. You may want to start with 5 topics, but adjust the number up or down depending on the word distributions you get. Decide on a suitable name for each topic. 


In [48]:
import os, csv, lda, nltk
import pandas as pd
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import PunktSentenceTokenizer,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [49]:
labels=df_final['google_labels']

In [64]:
import warnings
warnings.simplefilter("ignore")

ntopics= input('Provide the number of latent topics to be estimated: ')

def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(labels)

print(total_features_words.shape)

model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1, alpha = 0.5, eta = 0.005)
model.fit(total_features_words)

topic_word = model.topic_word_ 
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
topic_df=df_final.join(doc_topic)
natgeo_lda=pd.DataFrame()

for i in range(int(ntopics)):
    topic="topic_"+str(i)
    natgeo_lda[topic]=topic_df.groupby(topic_df["caption_new"])[i].mean()

natgeo_lda=natgeo_lda.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()
topics1.to_excel("topic_word_dist.xlsx")
natgeo_lda.to_excel("natgeo_topic_dist.xlsx")

Provide the number of latent topics to be estimated: 5


INFO:lda:n_documents: 874
INFO:lda:vocab_size: 1289
INFO:lda:n_words: 9865
INFO:lda:n_topics: 5
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -90848


(874, 1289)


INFO:lda:<10> log likelihood: -66927
INFO:lda:<20> log likelihood: -64765
INFO:lda:<30> log likelihood: -63967
INFO:lda:<40> log likelihood: -63474
INFO:lda:<50> log likelihood: -63180
INFO:lda:<60> log likelihood: -62969
INFO:lda:<70> log likelihood: -63003
INFO:lda:<80> log likelihood: -62916
INFO:lda:<90> log likelihood: -62575
INFO:lda:<100> log likelihood: -62578
INFO:lda:<110> log likelihood: -62571
INFO:lda:<120> log likelihood: -62482
INFO:lda:<130> log likelihood: -62472
INFO:lda:<140> log likelihood: -62429
INFO:lda:<150> log likelihood: -62280
INFO:lda:<160> log likelihood: -62255
INFO:lda:<170> log likelihood: -62195
INFO:lda:<180> log likelihood: -62155
INFO:lda:<190> log likelihood: -62164
INFO:lda:<200> log likelihood: -61974
INFO:lda:<210> log likelihood: -61780
INFO:lda:<220> log likelihood: -61573
INFO:lda:<230> log likelihood: -61612
INFO:lda:<240> log likelihood: -61550
INFO:lda:<250> log likelihood: -61545
INFO:lda:<260> log likelihood: -61471
INFO:lda:<270> log li

In [69]:
final_df_topics = df_final_final.merge(natgeo_lda)

In [ ]:
final_df_topics.to_excel("final_df_topics.xlsx")

Now sort the data from high to low engagement score, and take the highest and the lowest quartiles (by engagement score). What are the main differences in the average topic weights of images across the two quartiles (e.g., greater weight of some topics in the highest versus lowest engagement quartiles)? Show the main results in a table. 

This was done in an Excel spreadsheet and the results are summarized in the attached Word Doc.

### Task D - See Submitted Word Doc

What advice would you give Natgeo to increase engagement on its Instagram page based on your findings in Tasks B and C?   